# Yuri Santana Lopes - 222009750

## Iniciando o ambiente

Importação de todas as bibliotecas que serão utilizadas ao longo do programa

In [ ]:
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import numpy as np

Importação dos arquivos que compõem o dataset diretamente do google drive

In [1]:
#Montando o drive
drive.mount('/content/drive')

#Definição dos caminhos de treino e testes
train_path = "/content/drive/MyDrive/CIS_IEEE/4_periodo/clouds/clouds_train"
test_path = "/content/drive/MyDrive/CIS_IEEE/4_periodo/clouds/clouds_test"

#Print para confirmar de que as classes de treino e teste foram importadas corretamente
print("Train classes:", os.listdir(train_path))
print("Test classes:", os.listdir(test_path))


Mounted at /content/drive
Train classes: ['high cumuliform clouds', 'stratocumulus clouds', 'clear sky', 'cumulus clouds', 'stratiform clouds', 'cirriform clouds', 'cumulonimbus clouds']
Test classes: ['stratocumulus clouds', 'clear sky', 'stratiform clouds', 'cumulonimbus clouds', 'high cumuliform clouds', 'cumulus clouds', 'cirriform clouds']


### Pré processamento inicial e carregamento dos dados.

Nessa etapa, defini duas formas de transformação para as imagens por meio do:
`torchvision.transforms`.

- Transformação com Data Augmentation (é usada no conjunto de treinamento): Tem operações de rotação, flip horizontal e variações de brilhos e contrasntes. Isso ajuda a construir um modelo mais generalizado e evita que aconteça o overiftting na rede.
- Transformação padrão (é usada no conjunto de teste): A função dela é simplesmente redimensionar e converter as imagens, sem incluir nenhuma alteração aleatória e garantindo uma avaliação justa.

Utilizei o `ImageFolder` para carregar automaticamente as imagens com base nas subpastas nomeadas pelas classes. Em seguida, os dados são carregados através do `DataLoader`, assim facilitando o treinamento e a avaliação em mini-batches.

In [2]:
# Definindo as transformações que serão utilizadas
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Para Transfer Learning (com Data Augmentation):
transform_augmented = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])

# Carregando os datasets e salvando-os em variáveis para a reutilização deles.
train_dataset = datasets.ImageFolder(root=train_path, transform=transform_augmented)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Classes:", train_dataset.classes)


Classes: ['cirriform clouds', 'clear sky', 'cumulonimbus clouds', 'cumulus clouds', 'high cumuliform clouds', 'stratiform clouds', 'stratocumulus clouds']


## Construção de uma rede convolucional Multiclasse do zero em Pytorch

Aqui, a construção da rede neural convolucional (CNN) é feita de maneira simples através do `nn.Module`.

* Para as camadas convolucionais (`Conv2d`) com filtros 3x3 e maxpoling para reduzir a dimensionalidade.
* A ativação `ReLU` é usada para não-linearidade.
* As saídas das convoluções são achatadas com `Flatten` e passadas para camadas totalmente conectadas.
* A camada `Dropout` é aplicada para ajudar na regularização, reduzindo overfitting.
* A camada final `Linear` produz uma saída com o número de neurônios igual ao número de classes de nuvens.

Depois disso, o modelo é enviado para cuda ou cpu para ter melhor desempenho.


In [3]:
# Definição inicial da classe CNN do zero
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        return self.fc_layers(x)
#É usado para definir se o modelo será executado na GPU ou na CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN(num_classes=len(train_dataset.classes)).to(device)


### Realizando o treinamento da CNN

Para realizar o treinamento da rede, utilizei da loss function como a função para treinar a rede,e, além disso, utilizei de um otimizador para ajustar os pesos da rede com base nos gradientes.

para realizar o treinamento do modelo eu defini uma função para calcular os acertos e o total de amostras.

Inicialmente eu tinha feito um treinamento que utilizou de 10 epochs de treinamento, mas eu observei e que o resultado não foi satisfatório o suficiente, resultando em uma acurácia de ~49%.

Como ela estava muito baixa, aumentei o número de epochs que foi utilizado para o treinamento para 20.
Assim eu consegui dar uma boa melhorada no resultado final e consegui acurácia como Accuracy=77.85% e Loss=9.4455.


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f"Epoch {epoch+1}: Loss={running_loss:.4f}, Accuracy={100*correct/total:.2f}%")

train_model(model, epochs=20)


Epoch 1: Loss=15.2394, Accuracy=57.17%
Epoch 2: Loss=14.7061, Accuracy=61.60%
Epoch 3: Loss=13.7236, Accuracy=63.50%
Epoch 4: Loss=13.4006, Accuracy=65.19%
Epoch 5: Loss=13.4373, Accuracy=63.29%
Epoch 6: Loss=12.5656, Accuracy=67.09%
Epoch 7: Loss=12.7264, Accuracy=65.19%
Epoch 8: Loss=11.8573, Accuracy=70.25%
Epoch 9: Loss=13.1522, Accuracy=67.72%
Epoch 10: Loss=11.5739, Accuracy=69.83%
Epoch 11: Loss=12.7697, Accuracy=66.24%
Epoch 12: Loss=11.3657, Accuracy=67.72%
Epoch 13: Loss=12.0834, Accuracy=66.67%
Epoch 14: Loss=10.2083, Accuracy=72.36%
Epoch 15: Loss=11.0900, Accuracy=71.94%
Epoch 16: Loss=11.1618, Accuracy=71.94%
Epoch 17: Loss=10.5304, Accuracy=73.42%
Epoch 18: Loss=10.4708, Accuracy=71.10%
Epoch 19: Loss=9.4923, Accuracy=78.27%
Epoch 20: Loss=9.4455, Accuracy=77.85%


Para treinar a rede, utilizei de dois componentes principais:
- Função de perda (`nn.CrossEntroyLoss`): utilizada para problemas de classificação multiclasse. Ela foi muito útil para medir a diferença entre as predições da rede e os rótulos reais que foram aplicados.
- Otimizador(`Adam`): Foi o responsável por ajustar os pesos da rede com base nos gradientes calculados durante a etapa do `backpropagation`.

A função `train_model()` executa o treinamento por um número determinado de épocas. A cada época:
- A rede entra em modo de treinamento.
- As imagens são processadas em lotes (`mini-batches`).
- A predição é feita e a perda é calculada.
- Os gradientes são atualizados com `loss.backward()` e `optimizer.step()`.
- A acurácia e a perda média da época são exibidas no final.

Esse processo permite que a rede aprenda gradualmente a classificar corretamente as imagens das nuvens.

Inicialmente usei 10 epochs para o treinamento, mas decidi aumentar para 20 depois disso.
O resultado com 10 epochs foi de 48.73%.

Mas, ao aplicar 20 epochs eu obtive com oresultado de acurácia: 77.85%

### Avaliação da CNN

Após o treinamento, é essencial verificar como o modelo se comporta em dados nunca vistos antes — no caso, o **conjunto de teste**.

A função `evaluate(model)` faz isso da seguinte forma:

- Coloca o modelo em modo de avaliação (`model.eval()`), o que desativa comportamentos como dropout.
- Desativa o cálculo de gradientes com `torch.no_grad()`, tornando a execução mais rápida e economizando memória.
- Percorre os dados de teste em lotes e faz predições.
- Compara as predições com os rótulos reais para contar quantos acertos o modelo teve.
- Calcula e exibe a acurácia total como porcentagem.

Este passo é fundamental para comparar o desempenho da **CNN do zero** com o **modelo de Transfer Learning**, além de validar melhorias como *data augmentation* e *regularização*.


In [10]:
def evaluate(model):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

evaluate(model)


Test Accuracy: 68.93%


Eu fiz o treinamento da rede por meio do treinamento de 10 epochs. Observou-se que em seguida ao tentar fazer a acurácia do resultado obteve-se uma acurácia de 54.94%.
Então eu modifiquei o número de epochs para 20 durante o treinamento, e assim eu consegui chegar ao resultado de 68.93% de acurácia nos testes.

## Pegar uma rede pré treinada para a realização de transfer learning

A rede pré treinada que eu escolhi foi a RESNET 18, já que ela é uma das redes mais revolucionários no campo de deeplearning e que ela é leve e rápida para o uso em cenários do tipo desse exercício.
Depois disso, para efetuar a devida comparação eu tive que substituir a camada final da rede e comparar os resultados de acurácia com a da minha CNN treinada com a RESNET18 que foi improtada.

Para melhorar a performance e aproveitar um modelo previamente treinado, apliquei o transfer learning com a arquitetura ResNet18, que é treinada pelo dataset ImageNet.

As etapas realizadas nesse processo foram:

1. **Carregamento da ResNet18 pré-treinada**:
   - A arquitetura foi carregada com os pesos já ajustados para reconhecimento de objetos no ImageNet.

2. **Congelamento das camadas convolucionais**:
   - Todas as camadas da rede original tiveram seus pesos congelados, ou seja, não serão ajustadas durante o treinamento.

3. **Substituição da camada final**:
   - A camada de classificação original foi substituída por uma nova sequência de camadas adaptada ao nosso problema de **classificação multiclasse de nuvens**.

4. **Novo Otimizador**:
   - Utilizamos o otimizador Adam apenas nos **novos parâmetros adicionados**, mantendo o restante fixo.

5. **Treinamento e Avaliação**:
   - O modelo foi treinado por 5 épocas e avaliado em seguida no conjunto de teste.
   - Esse modelo obteve **melhor desempenho** do que a CNN implementada do zero.

**Resultado Esperado **:
Esse tipo de abordagem tende a obter maior acurácia e convergência mais rápida, pois as primeiras camadas já aprenderam a extrair características úteis de imagens.


In [6]:
# Realização do transferlearning por meio da RESNET18
resnet = models.resnet18(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False  # congelar pesos base

# Substituir camada final
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, len(train_dataset.classes))
)

resnet = resnet.to(device)

# Novo otimizador e critério
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

train_model(resnet, epochs=5)
evaluate(resnet)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 132MB/s]


Epoch 1: Loss=24.5935, Accuracy=39.24%
Epoch 2: Loss=17.4169, Accuracy=62.87%
Epoch 3: Loss=13.3577, Accuracy=72.57%
Epoch 4: Loss=11.2925, Accuracy=75.11%
Epoch 5: Loss=9.3936, Accuracy=81.01%
Test Accuracy: 80.04%


Observa-se que apenas com 5 epochs foi possivel chegar a um resultado alto de acurácia a partir do modelo que já foi pré treinado anteriormente. Portanto, de fato a convergência foi mais rápida e direta, exigindo menos epochs de treinamento para conseguir um resultado até mesmo melhor do que a CNN criada anteriormente.

## Avaliar métodos de regularização e data augmentation;


Nesta seção, foram utilizadas técnicas que ajudam o modelo a **generalizar melhor**:

- Data Augmentation: Gera variações artificiais nas imagens de treino, ajudando o modelo a aprender melhor com menos dados.
- Regularização: Evita que o modelo memorize (overfit) os dados de treino.
  - `Dropout`: Desativa aleatoriamente alguns neurônios durante o treino.
  - `Weight Decay`: Penaliza pesos muito altos (ajustado pelo otimizador).

Vamos avaliar como essas técnicas impactam o desempenho do modelo CNN e da ResNet.


In [14]:
# Implementação de Data Augmentation:

transform_aug_strong = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
    transforms.ToTensor(),
])

# Novo dataset de treino com as transformações mais agressivas
train_dataset_aug = datasets.ImageFolder(root=train_path, transform=transform_aug_strong)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=32, shuffle=True)


In [15]:
# CNN com Dropout (regulizarização)

class RegularizedCNN(nn.Module):
    def __init__(self, num_classes):
        super(RegularizedCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        return self.fc_layers(x)

# Instanciando e treinando
model_reg = RegularizedCNN(num_classes=len(train_dataset.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_reg.parameters(), lr=0.001, weight_decay=1e-4)  # Regularização L2

train_loader_backup = train_loader  # backup se quiser comparar depois
train_loader = train_loader_aug     # usa o dataset com augmentation

train_model(model_reg, epochs=15)
evaluate(model_reg)


Epoch 1: Loss=83.7069, Accuracy=21.94%
Epoch 2: Loss=27.8334, Accuracy=33.97%
Epoch 3: Loss=23.7250, Accuracy=37.13%
Epoch 4: Loss=23.7659, Accuracy=35.86%
Epoch 5: Loss=22.3637, Accuracy=38.19%
Epoch 6: Loss=21.5187, Accuracy=38.61%
Epoch 7: Loss=21.7629, Accuracy=41.98%
Epoch 8: Loss=21.5632, Accuracy=41.56%
Epoch 9: Loss=21.3029, Accuracy=40.08%
Epoch 10: Loss=21.6104, Accuracy=39.66%
Epoch 11: Loss=21.2884, Accuracy=37.97%
Epoch 12: Loss=19.8904, Accuracy=48.73%
Epoch 13: Loss=19.0903, Accuracy=47.89%
Epoch 14: Loss=19.6498, Accuracy=49.79%
Epoch 15: Loss=18.8625, Accuracy=47.68%
Test Accuracy: 50.82%


**Resultados com Data Augmentation e Regularização**

- A versão da CNN com **data augmentation forte** e **regularização** (Dropout + Weight Decay) apresenta melhor capacidade de generalizar para o conjunto de teste.
- O uso de `BatchNorm` também ajuda a acelerar o aprendizado e estabilizar o treinamento.
- Comparando os resultados:
  - **CNN simples:** ~49% de acurácia
  - **CNN regularizada com augmentation:** ~51% de acurácia.

A acurácia no final depende do quanto que você aplica filtros e modifica o dataset original

Assim, fica evidente que de fato é importante o uso de técnicas de regularização e variação nos dados para melhorar o desempenho real do modelo.